# Structured Streaming using the Python DataFrames API

Apache Spark includes a high-level stream processing API, [Structured Streaming](http://spark.apache.org/docs/latest/structured-streaming-programming-guide.html). In this notebook we take a quick look at how to use the DataFrame API to build Structured Streaming applications. We want to compute real-time metrics like running counts and windowed counts on a stream of timestamped actions (e.g. Open, Close, etc).

To run this notebook, import it and attach it to a Spark cluster.

## Sample Data
We have some sample action data as files in `/databricks-datasets/structured-streaming/events/` which we are going to use to build this appication. Let's take a look at the contents of this directory.

In [0]:
# Look at the content of the following folder: /databricks-datasets/structured-streaming/events/
# What do you see?

There are about 50 JSON files in the directory. Let's see what each JSON file contains.

In [0]:
# Look at the functions head in dbutils
# Open one file

Each line in the file contains JSON record with two fields - `time` and `action`. Let's try to analyze these files interactively.

## Batch/Interactive Processing
The usual first step in attempting to process the data is to interactively query the data. Let's define a static DataFrame on the files, and give it a table name.

In [0]:
from pyspark.sql.types import *

inputPath = "/databricks-datasets/structured-streaming/events/"

# Since we know the data format already, let's define the schema to speed up processing (no need for Spark to infer schema)
jsonSchema = StructType(
  [ StructField("time", TimestampType(), True),
   StructField("action", StringType(), True) ]
)



In [0]:
# Read all json files, taking into account the defined schema, and display the content 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, TimestampType, StringType

# Create a Spark session (only needed if running locally)
spark = SparkSession.builder.appName("StructuredStreaming").getOrCreate()

# Read the JSON files with the schema
df = spark.read.json(inputPath, schema=jsonSchema)

# Display the DataFrame
display(df)


time,action
2016-07-28T04:19:28.000+0000,Close
2016-07-28T04:19:28.000+0000,Close
2016-07-28T04:19:29.000+0000,Open
2016-07-28T04:19:31.000+0000,Close
2016-07-28T04:19:31.000+0000,Open
2016-07-28T04:19:31.000+0000,Open
2016-07-28T04:19:32.000+0000,Close
2016-07-28T04:19:33.000+0000,Close
2016-07-28T04:19:35.000+0000,Close
2016-07-28T04:19:36.000+0000,Open


- Compare the dates from the output without schema and with it. 
- Did you notice that inputPath is a folder?

In [0]:
# Calculate the total number of 'Open' and 'Close' actions 
from pyspark.sql.functions import col, count

# Count occurrences of 'Open' and 'Close' actions
action_counts = df.filter(col("action").isin(["Open", "Close"])) \
                  .groupBy("action") \
                  .agg(count("*").alias("total_count"))

# Display the result
display(action_counts)



action,total_count
Open,50000
Close,50000


In [0]:
# Determine min and max time
from pyspark.sql.functions import min, max

# Compute min and max time
time_range = df.agg(
    min("time").alias("min_time"),
    max("time").alias("max_time")
)

# Display the result
display(time_range)



min_time,max_time
2016-07-26T02:45:07.000+0000,2016-07-28T06:48:19.000+0000


In [0]:
# Calculate the number of "open" and "close" actions with one hour windows: staticCountsDF
# Look at groupBy(..., window) function
from pyspark.sql.functions import window, col, count

# Group by 1-hour time windows and count 'Open' and 'Close' actions
staticCountsDF = df.filter(col("action").isin(["Open", "Close"])) \
                   .groupBy(window(col("time"), "1 hour"), col("action")) \
                   .agg(count("*").alias("total_count")) \
                   .orderBy("window")

# Display the result
display(staticCountsDF)


window,action,total_count
"List(2016-07-26T02:00:00.000+0000, 2016-07-26T03:00:00.000+0000)",Open,179
"List(2016-07-26T02:00:00.000+0000, 2016-07-26T03:00:00.000+0000)",Close,11
"List(2016-07-26T03:00:00.000+0000, 2016-07-26T04:00:00.000+0000)",Close,344
"List(2016-07-26T03:00:00.000+0000, 2016-07-26T04:00:00.000+0000)",Open,1001
"List(2016-07-26T04:00:00.000+0000, 2016-07-26T05:00:00.000+0000)",Close,815
"List(2016-07-26T04:00:00.000+0000, 2016-07-26T05:00:00.000+0000)",Open,999
"List(2016-07-26T05:00:00.000+0000, 2016-07-26T06:00:00.000+0000)",Close,1003
"List(2016-07-26T05:00:00.000+0000, 2016-07-26T06:00:00.000+0000)",Open,1000
"List(2016-07-26T06:00:00.000+0000, 2016-07-26T07:00:00.000+0000)",Open,993
"List(2016-07-26T06:00:00.000+0000, 2016-07-26T07:00:00.000+0000)",Close,1011


In [0]:
# Make this window a sliding window (30 minutes overlap): staticCountsSW
from pyspark.sql.functions import window, col, count

# Group by a sliding window of 1 hour with a 30-minute overlap
staticCountsSW = df.filter(col("action").isin(["Open", "Close"])) \
                   .groupBy(window(col("time"), "1 hour", "30 minutes"), col("action")) \
                   .agg(count("*").alias("total_count")) \
                   .orderBy("window")

# Display the result
display(staticCountsSW)


window,action,total_count
"List(2016-07-26T02:00:00.000+0000, 2016-07-26T03:00:00.000+0000)",Open,179
"List(2016-07-26T02:00:00.000+0000, 2016-07-26T03:00:00.000+0000)",Close,11
"List(2016-07-26T02:30:00.000+0000, 2016-07-26T03:30:00.000+0000)",Close,116
"List(2016-07-26T02:30:00.000+0000, 2016-07-26T03:30:00.000+0000)",Open,680
"List(2016-07-26T03:00:00.000+0000, 2016-07-26T04:00:00.000+0000)",Close,344
"List(2016-07-26T03:00:00.000+0000, 2016-07-26T04:00:00.000+0000)",Open,1001
"List(2016-07-26T03:30:00.000+0000, 2016-07-26T04:30:00.000+0000)",Close,581
"List(2016-07-26T03:30:00.000+0000, 2016-07-26T04:30:00.000+0000)",Open,993
"List(2016-07-26T04:00:00.000+0000, 2016-07-26T05:00:00.000+0000)",Close,815
"List(2016-07-26T04:00:00.000+0000, 2016-07-26T05:00:00.000+0000)",Open,999


In [0]:
# Register staticCountsDF (createOrReplaceTempView) as table 'static_counts'
# Register staticCountsDF as a temporary view
staticCountsDF.createOrReplaceTempView("static_counts")

# Verify by running a SQL query
display(spark.sql("SELECT * FROM static_counts LIMIT 10"))


window,action,total_count
"List(2016-07-26T02:00:00.000+0000, 2016-07-26T03:00:00.000+0000)",Open,179
"List(2016-07-26T02:00:00.000+0000, 2016-07-26T03:00:00.000+0000)",Close,11
"List(2016-07-26T03:00:00.000+0000, 2016-07-26T04:00:00.000+0000)",Close,344
"List(2016-07-26T03:00:00.000+0000, 2016-07-26T04:00:00.000+0000)",Open,1001
"List(2016-07-26T04:00:00.000+0000, 2016-07-26T05:00:00.000+0000)",Close,815
"List(2016-07-26T04:00:00.000+0000, 2016-07-26T05:00:00.000+0000)",Open,999
"List(2016-07-26T05:00:00.000+0000, 2016-07-26T06:00:00.000+0000)",Open,1000
"List(2016-07-26T05:00:00.000+0000, 2016-07-26T06:00:00.000+0000)",Close,1003
"List(2016-07-26T06:00:00.000+0000, 2016-07-26T07:00:00.000+0000)",Close,1011
"List(2016-07-26T06:00:00.000+0000, 2016-07-26T07:00:00.000+0000)",Open,993


Now we can directly use SQL to query the table.

In [0]:
%sql
-- Count all Open and Close actions in the table static_counts  
SELECT action, SUM(total_count) AS total_count
FROM static_counts
WHERE action IN ('Open', 'Close')
GROUP BY action;


action,total_count
Open,50000
Close,50000


In [0]:
%sql
-- How many actions (Close and Open separately) is within each time window (in the table static_counts)
-- Make a plot
SELECT window.start AS window_start,
       window.end AS window_end,
       action,
       SUM(total_count) AS action_count
FROM static_counts
WHERE action IN ('Open', 'Close')
GROUP BY window.start, window.end, action
ORDER BY window_start;


window_start,window_end,action,action_count
2016-07-26T02:00:00.000+0000,2016-07-26T03:00:00.000+0000,Open,179
2016-07-26T02:00:00.000+0000,2016-07-26T03:00:00.000+0000,Close,11
2016-07-26T03:00:00.000+0000,2016-07-26T04:00:00.000+0000,Close,344
2016-07-26T03:00:00.000+0000,2016-07-26T04:00:00.000+0000,Open,1001
2016-07-26T04:00:00.000+0000,2016-07-26T05:00:00.000+0000,Close,815
2016-07-26T04:00:00.000+0000,2016-07-26T05:00:00.000+0000,Open,999
2016-07-26T05:00:00.000+0000,2016-07-26T06:00:00.000+0000,Close,1003
2016-07-26T05:00:00.000+0000,2016-07-26T06:00:00.000+0000,Open,1000
2016-07-26T06:00:00.000+0000,2016-07-26T07:00:00.000+0000,Open,993
2016-07-26T06:00:00.000+0000,2016-07-26T07:00:00.000+0000,Close,1011


Note the two ends of the graph. The close actions are generated such that they are after the corresponding open actions, so there are more "opens" in the beginning and more "closes" in the end.

## Demo: Stream Processing 
Now that we have analyzed the data interactively, let's convert this to a streaming query that continuously updates as data comes. Since we just have a static set of files, we are going to emulate a stream from them by reading one file at a time, in the chronological order they were created. The query we have to write is pretty much the same as the interactive query above.

In [0]:
from pyspark.sql.functions import *

# Read data from a file
# Similar to definition of staticInputDF above, just using `readStream` instead of `read`
streamingInputDF = (
  spark
    .readStream                       
    .schema(jsonSchema)               # Set the schema of the JSON data
    .option("maxFilesPerTrigger", 1)  # Treat a sequence of files as a stream by picking one file at a time
    .json(inputPath)
)

# Do some transformations
# Same query as staticInputDF
streamingCountsDF = (                 
  streamingInputDF
    .groupBy(
      streamingInputDF.action, 
      window(streamingInputDF.time, "1 hour"))
    .count()
)

# Is this DF actually a streaming DF?
streamingCountsDF.isStreaming

Out[12]: True

In [0]:
# Display input data
streamingInputDF.display()

time,action
2016-07-26T02:45:07.000+0000,Open
2016-07-26T02:45:47.000+0000,Open
2016-07-26T02:46:42.000+0000,Open
2016-07-26T02:46:59.000+0000,Open
2016-07-26T02:47:05.000+0000,Open
2016-07-26T02:47:14.000+0000,Open
2016-07-26T02:47:25.000+0000,Open
2016-07-26T02:47:26.000+0000,Open
2016-07-26T02:47:28.000+0000,Open
2016-07-26T02:47:36.000+0000,Open


In [0]:
# Display transformed data
streamingCountsDF.display()

action,window,count
Close,"List(2016-07-27T00:00:00.000+0000, 2016-07-27T01:00:00.000+0000)",1008
Open,"List(2016-07-26T22:00:00.000+0000, 2016-07-26T23:00:00.000+0000)",997
Open,"List(2016-07-27T14:00:00.000+0000, 2016-07-27T15:00:00.000+0000)",984
Close,"List(2016-07-26T14:00:00.000+0000, 2016-07-26T15:00:00.000+0000)",994
Open,"List(2016-07-28T04:00:00.000+0000, 2016-07-28T05:00:00.000+0000)",825
Close,"List(2016-07-27T14:00:00.000+0000, 2016-07-27T15:00:00.000+0000)",948
Close,"List(2016-07-26T23:00:00.000+0000, 2016-07-27T00:00:00.000+0000)",950
Open,"List(2016-07-27T06:00:00.000+0000, 2016-07-27T07:00:00.000+0000)",1016
Open,"List(2016-07-28T03:00:00.000+0000, 2016-07-28T04:00:00.000+0000)",996
Close,"List(2016-07-26T04:00:00.000+0000, 2016-07-26T05:00:00.000+0000)",815


In [0]:
# Add aditional filter to transformed dataframe
streamingCountsDF.filter(streamingCountsDF.action == 'Open').display()

action,window,count
Open,"List(2016-07-26T12:00:00.000+0000, 2016-07-26T13:00:00.000+0000)",996
Open,"List(2016-07-27T11:00:00.000+0000, 2016-07-27T12:00:00.000+0000)",998
Open,"List(2016-07-27T01:00:00.000+0000, 2016-07-27T02:00:00.000+0000)",1004
Open,"List(2016-07-26T22:00:00.000+0000, 2016-07-26T23:00:00.000+0000)",997
Open,"List(2016-07-26T23:00:00.000+0000, 2016-07-27T00:00:00.000+0000)",1000
Open,"List(2016-07-27T10:00:00.000+0000, 2016-07-27T11:00:00.000+0000)",1006
Open,"List(2016-07-27T22:00:00.000+0000, 2016-07-27T23:00:00.000+0000)",986
Open,"List(2016-07-26T13:00:00.000+0000, 2016-07-26T14:00:00.000+0000)",1006
Open,"List(2016-07-27T14:00:00.000+0000, 2016-07-27T15:00:00.000+0000)",984
Open,"List(2016-07-27T18:00:00.000+0000, 2016-07-27T19:00:00.000+0000)",995


As you can see, `streamingCountsDF` is a streaming Dataframe (`streamingCountsDF.isStreaming` was `true`). You can start streaming computation, by defining the sink and starting it. 
In our case, we want to interactively query the counts (same queries as above), so we will set the complete set of 1 hour counts to be in a in-memory table.

In [0]:
spark.conf.set("spark.sql.shuffle.partitions", "2")  # keep the size of shuffles small

query = (
  streamingCountsDF
    .writeStream
    .format("memory")        # memory = store in-memory table 
    .queryName("counts")     # counts = name of the in-memory table
    .outputMode("complete")  # complete = all the counts should be in the table
    #.trigger(processingTime='15 seconds')
    .start()
)

`query` is a handle to the streaming query that is running in the background. This query is continuously picking up files and updating the windowed counts. 

Note the status of query in the above cell. The progress bar shows that the query is active. 
Furthermore, if you expand the `> counts` above, you will find the number of files they have already processed. 

Let's wait a bit for a few files to be processed and then interactively query the in-memory `counts` table.

In [0]:
%sql
SELECT *
FROM counts

action,window,count
Close,"List(2016-07-26T06:00:00.000+0000, 2016-07-26T07:00:00.000+0000)",328
Open,"List(2016-07-26T02:00:00.000+0000, 2016-07-26T03:00:00.000+0000)",179
Close,"List(2016-07-26T04:00:00.000+0000, 2016-07-26T05:00:00.000+0000)",815
Close,"List(2016-07-26T05:00:00.000+0000, 2016-07-26T06:00:00.000+0000)",1003
Close,"List(2016-07-26T02:00:00.000+0000, 2016-07-26T03:00:00.000+0000)",11
Open,"List(2016-07-26T06:00:00.000+0000, 2016-07-26T07:00:00.000+0000)",320
Open,"List(2016-07-26T03:00:00.000+0000, 2016-07-26T04:00:00.000+0000)",1001
Close,"List(2016-07-26T03:00:00.000+0000, 2016-07-26T04:00:00.000+0000)",344
Open,"List(2016-07-26T04:00:00.000+0000, 2016-07-26T05:00:00.000+0000)",999
Open,"List(2016-07-26T05:00:00.000+0000, 2016-07-26T06:00:00.000+0000)",1000


In [0]:
from time import sleep
sleep(5)  # wait a bit for computation to start

In [0]:
%sql
select action, date_format(window.end, "MMM-dd HH:mm") as time, count
from counts
order by time, action

action,time,count
Close,Jul-26 03:00,11
Open,Jul-26 03:00,179
Close,Jul-26 04:00,344
Open,Jul-26 04:00,1001
Close,Jul-26 05:00,815
Open,Jul-26 05:00,999
Close,Jul-26 06:00,1003
Open,Jul-26 06:00,1000
Close,Jul-26 07:00,1011
Open,Jul-26 07:00,993


We see the timeline of windowed counts (similar to the static one earlier) building up. If we keep running this interactive query repeatedly, we will see the latest updated counts which the streaming query is updating in the background.

In [0]:
sleep(5)  # wait a bit more for more data to be computed

In [0]:
%sql select action, date_format(window.end, "MMM-dd HH:mm") as time, count from counts order by time, action

action,time,count
Close,Jul-26 03:00,11
Open,Jul-26 03:00,179
Close,Jul-26 04:00,344
Open,Jul-26 04:00,1001
Close,Jul-26 05:00,815
Open,Jul-26 05:00,999
Close,Jul-26 06:00,1003
Open,Jul-26 06:00,1000
Close,Jul-26 07:00,1011
Open,Jul-26 07:00,993


In [0]:
sleep(5)  # wait a bit more for more data to be computed

In [0]:
%sql select action, date_format(window.end, "MMM-dd HH:mm") as time, count from counts order by time, action

action,time,count
Close,Jul-26 03:00,11
Open,Jul-26 03:00,179
Close,Jul-26 04:00,344
Open,Jul-26 04:00,1001
Close,Jul-26 05:00,815
Open,Jul-26 05:00,999
Close,Jul-26 06:00,1003
Open,Jul-26 06:00,1000
Close,Jul-26 07:00,1011
Open,Jul-26 07:00,993


Also, let's see the total number of "opens" and "closes".

In [0]:
%sql 
select action, sum(count) as total_count 
from counts 
group by action 
order by action

action,total_count
Close,29490
Open,30510


If you keep running the above query repeatedly, you will always find that the number of "opens" is more than the number of "closes", as expected in a data stream where a "close" always appear after corresponding "open". This shows that Structured Streaming ensures **prefix integrity**. Read the blog posts linked below if you want to know more.

Note that there are only a few files, so consuming all of them there will be no updates to the counts. Rerun the query if you want to interact with the streaming query again.

Finally, you can stop the query running in the background, either by clicking on the 'Cancel' link in the cell of the query, or by executing `query.stop()`. Either way, when the query is stopped, the status of the corresponding cell above will automatically update to `TERMINATED`.

In [0]:
query.stop()

### IoT data

Develop a streaming example on `IoT device`dataset:

- inspect the dataset
- ask yourself couple of questions about the data and try to answer them (eg. how many steps users do, how many calories do they burn...)
- you read the data in streaming fashion (file by file) and keep the data for only one company? Here are some hints:
  - you can find the schema in the readme file 
  - as above, use this option: .option("maxFilesPerTrigger", 1)
  - use user_id or device_id for grouping
  - use timestamp for window definition
  - you can try streaming joins with the user data (/databricks-datasets/iot-stream/data-user/userData.csv). Here is the doc: https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#join-operations

In [0]:
# Iot stream dataset
display(dbutils.fs.ls('/databricks-datasets/iot-stream/'))

path,name,size,modificationTime
dbfs:/databricks-datasets/iot-stream/README.md,README.md,1596,1532465738000
dbfs:/databricks-datasets/iot-stream/data-device/,data-device/,0,0
dbfs:/databricks-datasets/iot-stream/data-user/,data-user/,0,0


In [0]:
# Read the README file
# List the contents of the iot-stream directory
files = dbutils.fs.ls('/databricks-datasets/iot-stream/')
spark = SparkSession.builder.appName("PreviewReadmeFile").getOrCreate()
# Define the schema (copy from the README) data-device
readme_schema = StructType([
    StructField("line", StringType(), True)
])
for file in files:
    print(file.path)


dbfs:/databricks-datasets/iot-stream/README.md
dbfs:/databricks-datasets/iot-stream/data-device/
dbfs:/databricks-datasets/iot-stream/data-user/


In [0]:
# Define the schema (copy from the README) data-device
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType

# Define the schema
iot_schema = StructType([
    StructField("device_id", StringType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("temperature", DoubleType(), True),
    StructField("humidity", DoubleType(), True),
    StructField("location", StringType(), True),
    StructField("status", StringType(), True)
])

# Show the schema
iot_schema


Out[46]: StructType([StructField('device_id', StringType(), True), StructField('timestamp', TimestampType(), True), StructField('temperature', DoubleType(), True), StructField('humidity', DoubleType(), True), StructField('location', StringType(), True), StructField('status', StringType(), True)])

In [0]:
# Open one file to see how the data looks like (as a static dataframe)
# List the contents of the iot-stream directory
files = dbutils.fs.ls('/databricks-datasets/iot-stream/')
for file in files:
    print(file.path)


dbfs:/databricks-datasets/iot-stream/README.md
dbfs:/databricks-datasets/iot-stream/data-device/
dbfs:/databricks-datasets/iot-stream/data-user/


In [0]:
# Define your streaming dataframe
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType

# Define the schema for the IoT data (you may need to adjust this based on your dataset)
iot_schema = StructType([
    StructField("device_id", StringType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("temperature", DoubleType(), True),
    StructField("humidity", DoubleType(), True),
    StructField("location", StringType(), True),
    StructField("status", StringType(), True)
])

# Define the streaming DataFrame by reading CSV files from a directory
streaming_df = spark.readStream.schema(iot_schema).csv("/databricks-datasets/iot-stream/")

# Show the schema of the streaming DataFrame
streaming_df.printSchema()


root
 |-- device_id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- temperature: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- location: string (nullable = true)
 |-- status: string (nullable = true)



In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, col
from pyspark.sql.types import StructType, StructField, StringType

# Initialize Spark session
spark = SparkSession.builder.appName("WordCount").getOrCreate()

# Path to the files
files = dbutils.fs.ls('/databricks-datasets/iot-stream/')

# For reading plain text files (no schema required)
static_df = spark.read.text(files)

# Show the first few lines of the text file
static_df.show(truncate=False)

# Get the total number of lines in the file
total_lines = static_df.count()
print(f"Total lines in the file: {total_lines}")

# For streaming data (assuming you want to process the file in real-time)
streaming_df = spark.readStream.text(files)

# Split the lines into words
words_df = streaming_df.select(explode(split(col("value"), " ")).alias("word"))

# Group by word and count occurrences
word_counts = words_df.groupBy("word").count()

# Example of starting the stream (outputting to console)
query = word_counts.writeStream.outputMode("complete").format("console").start()

# Wait for the streaming query to finish
query.awaitTermination()


In [0]:
query = word_counts.writeStream.outputMode("complete").format("console").start()